# Data processing
---

Here we pre-process some of the data for single Helium stars that run on Condor

## Step 1: Copy the data from the server to a local machine
---

We are going to use the script "fetch_data.py". 

Make sure that the script is updated with the correct path to the server where the data are stored -> grid_dir

> If you want all of the data to be copied, open a terminal window and type:

```bash
    ~$ python fetch_data.py /path/to/local/folder/name True
```

> If you want to obtain only a copy of the history file, and the final/last saved profile, type:

```bash
    ~$ python fetch_data.py /path/to/local/folder/name False
```

## Step 2: Create a csv file with data relevant to core growth
---

Here we are going to write the data from our local machine into a csv file
that contains information for the initial mass, initial metallicity, overshooting factor,
final core mass, and final envelope mass.

For this purpose, we're exploiting the **[mesa_star_class]** and the getCoreMass() method.

## Step 3: Read the csv file and plot the data
---

Make plots like **initial_mass vs final_core_mass** by keeping the metallicity constant and varying only
the overshooting factor.

Plot **initial_mass vs final_envelope_mass** by keeping the overshooting factor constant and varying only
the initial metallicity. Do the same for the **final_core_mass** to explore the dependency og core growth 
on metallicity.



In [ ]:
# Import modules

import numpy as np
import mesa_reader as mr
import mesaPlot as mp
import glob, os, re, csv
from mesa_star_class import MESA_STAR as ms
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap as cmap
from matplotlib.colors import BoundaryNorm as bn
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import scipy.optimize as so
import math
import homogenize_plot as hp
%matplotlib inline

In [ ]:
# Set the path where our data are located in the local machine
full_grid = '/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data'

# Set path where plots will be saved
plot_out = '/vol/aibn1107/data2/schanlar/HeCoresCondor/Plots'

In [ ]:
# Create a function that loops over the data and returns a zip object
# The output of this function will be used to write the zipped data
# in a csv file

def coreGrowth(data_dir):
    
    initial_mass, initial_metallicity, overshooting_factor = [], [], []

    final_core_mass, final_envelope_mass = [], []
    
    termination_code = []
    
    if os.path.isdir(data_dir):
        pass
    else:
        raise SystemExit('This directory does not exist!')

    for path in glob.glob(f'{data_dir}/*'):

        string_input = path.split('/')[-1]
        logs_path = os.path.join(path, 'LOGS')

        if ms.name_is_valid(string_input):
            
            try:
                
                os.system(f'cd {logs_path}')

                # Find the last saved profile from the profiles.index 
                prof_num = ms.find_profile_number(logs_path, num = -1)
                
                # Find the termination code for this model
                n = ms.find_termination_code(f'{path}/condor.out')

                # Define a MESA_STAR object
                star = ms.from_string(string_input,
                                     history_path = logs_path,
                                     profile_path = logs_path,
                                     profile_number = prof_num)
                
                
                # Estimate the core mass for the star
                i, j, k, l, m = star.getCoreMass()
                
                initial_mass.append(i)
                initial_metallicity.append(j)
                overshooting_factor.append(k)
                final_core_mass.append(l)
                final_envelope_mass.append(m)
                termination_code.append(n)
                
                print(f'Model {string_input}: {i, j, k, l, m, n}')
                
            except:
                print(f'Something went wrong when trying to load the data from {path}')

            
        else:
            print(f'Name is not valid for {path}')
            
    return zip(initial_mass, initial_metallicity, overshooting_factor, final_core_mass, final_envelope_mass, termination_code)



In [ ]:
# Examine the data when necessary and create the csv file to store them

zipped_results = coreGrowth(full_grid)

ms.export_csvFile(zipped_results, name = 'coreGrowthData', termination=True)

In [ ]:
# Read, parse, and sort the data according to metallicity and overshooting factor

mass,metal,fover,mcore,menv,ffate = [],[],[],[],[],[]

massLM_WNO, massIM_WNO, massSM_WNO = [], [], []
massLM_WO1, massIM_WO1, massSM_WO1 = [], [], []
massLM_WO2, massIM_WO2, massSM_WO2 = [], [], []

coreMassLM_WNO, coreMassIM_WNO, coreMassSM_WNO = [], [], []
coreMassLM_WO1, coreMassIM_WO1, coreMassSM_WO1 = [], [], []
coreMassLM_WO2, coreMassIM_WO2, coreMassSM_WO2 = [], [], []

envMassLM_WNO, envMassIM_WNO, envMassSM_WNO = [], [], []
envMassLM_WO1, envMassIM_WO1, envMassSM_WO1 = [], [], []
envMassLM_WO2, envMassIM_WO2, envMassSM_WO2 = [], [], []

with open('fullCoreGrowthData.csv', 'r') as f:
    # Skip header of first row
    next(f)
    r = csv.reader(f)
    for row in r:
        mass.append(row[0])
        metal.append(row[1])
        fover.append(row[2])
        mcore.append(row[3])
        menv.append(row[4])
        ffate.append(row[5])
        
for i,j,k,l,m,n in list(zip(mass, metal, fover, mcore, menv, ffate)):
    
        # Consider only data than retain an envelope mass < 0.3 Msol, 
        # or their core mass exceed the Chandrasekhar mass limit, 
        # or they've formed a silicon core/shell at the time they
        # were terminated. In the latter case, we assume these models
        # will eventually grow to Chandra limit and experience a CCSN
        if float(m) < 0.3 or float(l) >= 1.45 or n == 'SiCORE' or n == 'SiCORE*' or n == 'SiCORE**'or \
                n == 'ONeSiWD' or n == 'ONeSiWD*' or n == 'ONeSiWD**':
            
            if j == '0.0001' and k == '0.0000':
                massLM_WNO.append(float(i))
                coreMassLM_WNO.append(float(l))
                envMassLM_WNO.append(float(m))

            elif j == '0.0010' and k == '0.0000':
                massIM_WNO.append(float(i))
                coreMassIM_WNO.append(float(l))
                envMassIM_WNO.append(float(m))

            elif j == '0.0200' and k == '0.0000':
                massSM_WNO.append(float(i))
                coreMassSM_WNO.append(float(l))
                envMassSM_WNO.append(float(m))

            elif j == '0.0001' and k == '0.0140':
                massLM_WO1.append(float(i))
                coreMassLM_WO1.append(float(l))
                envMassLM_WO1.append(float(m))

            elif j == '0.0010' and k == '0.0140':
                massIM_WO1.append(float(i))
                coreMassIM_WO1.append(float(l))
                envMassIM_WO1.append(float(m))

            elif j == '0.0200' and k == '0.0140':
                massSM_WO1.append(float(i))
                coreMassSM_WO1.append(float(l))
                envMassSM_WO1.append(float(m))

            elif j == '0.0001' and k == '0.0160':
                massLM_WO2.append(float(i))
                coreMassLM_WO2.append(float(l))
                envMassLM_WO2.append(float(m))

            elif j == '0.0010' and k == '0.0160':
                massIM_WO2.append(float(i))
                coreMassIM_WO2.append(float(l))
                envMassIM_WO2.append(float(m))

            elif j == '0.0200' and k == '0.0160':
                massSM_WO2.append(float(i))
                coreMassSM_WO2.append(float(l))
                envMassSM_WO2.append(float(m))
            
            
    
                

In [ ]:
def poly_fit(x, a, b, c):
    '''
    Normal cubic function y = x**3 flattens
    out at (0,0).
    Here, we introduce a shift in x,y
    '''
    fun = a * (x - b)**3 + c
    return fun

def exp_fit(x, a, b):
    fun = a * (1 - np.exp( -b * x))
    return fun

def piecewise_linear(x, a0, a1, xm):
    return np.piecewise(x, [x < xm, x >= xm], [lambda x: a0 + a1 * x, lambda x:a0 + a1 * xm])

In [ ]:
hp.homogenise_plot(fig_width = 10, fig_height = 10, columns = 2, fontsize = 20)

'''
# For interpolated data
newx_SMWNO, newy_SMWNO = [], []
newx_LMWNO, newy_LMWNO = [], []
newx_IMWNO, newy_IMWNO = [], []


# Fitting parameters
# -------------------------------------------------------------
popt_SMWNO, pcov_SMWNO = so.curve_fit(poly_fit, massSM_WNO, coreMassSM_WNO)
print(popt_SMWNO)
print("a = %f +/- %f" % (popt_SMWNO[0], np.sqrt(pcov_SMWNO[0][0])))
print("b = %f +/- %f" % (popt_SMWNO[1], np.sqrt(pcov_SMWNO[1][1])))
print("c = %f +/- %f" % (popt_SMWNO[2], np.sqrt(pcov_SMWNO[2][2])))
x_fit = np.linspace(0.8, 3.5, 28)
y_fit_SMWNO = poly_fit(x_fit, *popt_SMWNO)

popt_LMWNO, pcov_LMWNO = so.curve_fit(poly_fit, massLM_WNO, coreMassLM_WNO)
print(popt_LMWNO)
print("a = %f +/- %f" % (popt_LMWNO[0], np.sqrt(pcov_LMWNO[0][0])))
print("b = %f +/- %f" % (popt_LMWNO[1], np.sqrt(pcov_LMWNO[1][1])))
print("c = %f +/- %f" % (popt_LMWNO[2], np.sqrt(pcov_LMWNO[2][2])))

y_fit_LMWNO = poly_fit(x_fit, *popt_LMWNO)

popt_IMWNO, pcov_IMWNO = so.curve_fit(poly_fit, massIM_WNO, coreMassIM_WNO)
print(popt_IMWNO)
print("a = %f +/- %f" % (popt_IMWNO[0], np.sqrt(pcov_IMWNO[0][0])))
print("b = %f +/- %f" % (popt_IMWNO[1], np.sqrt(pcov_IMWNO[1][1])))
print("c = %f +/- %f" % (popt_IMWNO[2], np.sqrt(pcov_IMWNO[2][2])))

y_fit_IMWNO = poly_fit(x_fit, *popt_IMWNO)


# Interpolated data points
# -------------------------------------------------------------
for i in x_fit:
    found = False
    for j in massSM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_SMWNO.append(i)
        newy_SMWNO.append(poly_fit(i, *popt_SMWNO))
        
for i in x_fit:
    found = False
    for j in massLM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_LMWNO.append(i)
        #newy_LMWNO.append(poly_fit(i, 0.2, 2.22, 1.35))
        newy_LMWNO.append(poly_fit(i, *popt_LMWNO))
        
for i in x_fit:
    found = False
    for j in massIM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_IMWNO.append(i)
        #newy_IMWNO.append(poly_fit(i, 0.2, 2.25, 1.36))
        newy_IMWNO.append(poly_fit(i, *popt_IMWNO))
# -------------------------------------------------------------
'''

# Plot
# -------------------------------------------------------------

fig, ax = plt.subplots()
ax.set_xlabel(r'Initial Mass [M$_{\odot}$]')
ax.set_ylabel(r'Final Core Mass [M$_{\odot}$]')

ax.set_xlim(0.5, 3.6)
ax.set_ylim(0.5, 1.8)
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))

ax.scatter(massLM_WNO, coreMassLM_WNO, facecolors = 'none', edgecolors = 'g', marker = 'd', 
           s = 150, label = r'series $Z = 0.0001; f = 0.0$')
ax.scatter(massIM_WNO, coreMassIM_WNO, facecolors = 'none', edgecolors = 'b', marker = 's', 
           s = 150, label = r'series $Z = 0.001; f = 0.0$')
ax.scatter(massSM_WNO, coreMassSM_WNO, facecolors = 'none', edgecolors = 'r', marker = 'o', 
           s = 150, label = r'series $Z = 0.02; f = 0.0$')

#ax.scatter(massLM_WO1, coreMassLM_WO1, c = 'g', marker = 'D', label = 'series LM;WO1')
#ax.scatter(massIM_WO1, coreMassIM_WO1, c = 'b', marker = 's', label = 'series IM;WO1')
#ax.scatter(massSM_WO1, coreMassSM_WO1, c = 'r', marker = 'o', label = 'series SM;WO1')

#ax.scatter(massLM_WO2, coreMassLM_WO2, c = 'g', marker = 'D', label = 'series LM;WO2')
#ax.scatter(massIM_WO2, coreMassIM_WO2, c = 'b', marker = 's', label = 'series IM;WO2')
#ax.scatter(massSM_WO2, coreMassSM_WO2, c = 'r', marker = 'o', label = 'series SM;WO2')

legend = ax.legend(loc = 'lower right', shadow = True, prop={'size':20})
# -------------------------------------------------------------



# Fitting curves + interpolated data
# -------------------------------------------------------------

#ax.plot(x_fit, y_fit_LMWNO, c = 'g', linestyle = '--')
#ax.scatter(newx_LMWNO, newy_LMWNO, facecolors = 'none', edgecolors = 'g', marker = 'D')

#ax.plot(x_fit, y_fit_IMWNO, c = 'b', linestyle = '--')
#ax.scatter(newx_IMWNO, newy_IMWNO, facecolors = 'none', edgecolors = 'b', marker = 's')

#ax.plot(x_fit, y_fit_SMWNO, c = 'r', linestyle = '--')
#ax.scatter(newx_SMWNO, newy_SMWNO, facecolors = 'none', edgecolors = 'r', marker = 'o')
# -------------------------------------------------------------




'''
# Final fate regions
# -------------------------------------------------------------
x_fill_between_12 = np.linspace(0.5, 1.3, 6)
y_fill_between_1 = poly_fit(x_fill_between_12, 0.18 , 2.18, 1.4)
y_fill_between_2 = poly_fit(x_fill_between_12, 0.2 , 2.35, 1.25)

x_fill_between_34 = np.linspace(1.3, 1.7, 5)
y_fill_between_3 = poly_fit(x_fill_between_34, 0.18 , 2.18, 1.4)
y_fill_between_4 = poly_fit(x_fill_between_34, 0.2 , 2.35, 1.25)

x_fill_between_56 = np.linspace(1.7, 2.0, 4)
y_fill_between_5 = poly_fit(x_fill_between_56, 0.18 , 2.18, 1.4)
y_fill_between_6 = poly_fit(x_fill_between_56, 0.2 , 2.35, 1.25)

x_fill_between_78 = np.linspace(2.0, 2.8, 9)
y_fill_between_7 = poly_fit(x_fill_between_78, 0.18 , 2.18, 1.4)
y_fill_between_8 = poly_fit(x_fill_between_78, 0.2 , 2.35, 1.25)

x_fill_between_910 = np.linspace(2.8, 3.6, 8)
y_fill_between_9 = poly_fit(x_fill_between_910, 0.18 , 2.18, 1.4)
y_fill_between_10 = poly_fit(x_fill_between_910, 0.2 , 2.35, 1.25)

ax.fill_between(x_fill_between_12, y_fill_between_1, y_fill_between_2, color = 'g', alpha = 0.1)
ax.fill_between(x_fill_between_34, y_fill_between_3, y_fill_between_4, color = 'y', alpha = 0.1)
ax.fill_between(x_fill_between_56, y_fill_between_5, y_fill_between_6, color = 'purple', alpha = 0.1)
ax.fill_between(x_fill_between_78, y_fill_between_7, y_fill_between_8, facecolor = 'none', hatch = 'X',
                edgecolor="purple", alpha = 0.3)
ax.fill_between(x_fill_between_910, y_fill_between_9, y_fill_between_10, color = 'r', alpha = 0.1)
'''


#plt.savefig('coreGrowth_WNO.pdf')

# revert changes to the configuration made by homogenize_plot:
#hp.revert_params()

In [ ]:
hp.homogenise_plot(fig_width = 10, fig_height = 10, columns = 2, fontsize = 20)

'''
# For interpolated data
newx_SMWO1, newy_SMWO1 = [], []
newx_LMWO1, newy_LMWO1 = [], []
newx_IMWO1, newy_IMWO1 = [], []


# Fitting parameters
# -------------------------------------------------------------
popt, pcov = so.curve_fit(exp_fit, massSM_WO1[:12], coreMassSM_WO1[:12])
print(popt)
print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))
#print("c = %f +/- %f" % (popt[2], np.sqrt(pcov[2][2])))
x_fit_SMWO1 = np.linspace(0.8, 2.8, 21)
#y_fit_SMWO1 = exp_fit(x_fit_SMWO1, 2.4, 1.17)

popt, pcov = so.curve_fit(exp_fit, massLM_WO1[:10], coreMassLM_WO1[:10])
print(popt)
print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))
#print("c = %f +/- %f" % (popt[2], np.sqrt(pcov[2][2])))
x_fit_LMWO1 = np.linspace(0.8, 1.6, 9)
#y_fit_LMWNO = exp_fit(x_fit_LMWO1, 0.2, 2.145, 1.289)

popt, pcov = so.curve_fit(exp_fit, massIM_WNO[:10], coreMassIM_WNO[:10])
print(popt)
print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))
#print("c = %f +/- %f" % (popt[2], np.sqrt(pcov[2][2])))
x_fit_IMWO1 = np.linspace(0.8, 1.6, 9)
#y_fit_IMWO1 = exp_fit(x_fit_IMWO1, 0.2, 2.206, 1.32)


# Interpolated data points
# -------------------------------------------------------------
for i in x_fit:
    found = False
    for j in massSM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_SMWO1.append(i)
        newy_SMWO1.append(poly_fit(i, 0.2, 2.32, 1.34))
        
for i in x_fit:
    found = False
    for j in massLM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_LMWO1.append(i)
        newy_LMWO1.append(poly_fit(i, 0.2, 2.145, 1.289))
        
for i in x_fit:
    found = False
    for j in massIM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_IMWO1.append(i)
        newy_IMWO1.append(poly_fit(i, 0.2, 2.206, 1.32))
# -------------------------------------------------------------
'''

# Plot
# -------------------------------------------------------------

fig, ax = plt.subplots()
ax.set_xlabel(r'Initial Mass [M$_{\odot}$]')
ax.set_ylabel(r'Final Core Mass [M$_{\odot}$]')

ax.set_xlim(0.5, 3.6)
ax.set_ylim(0.5, 1.8)
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))


ax.scatter(massLM_WO1, coreMassLM_WO1, facecolors = 'none', edgecolors = 'g', marker = 'd', 
           s= 150, label = r'series Z = 0.0001; f = 0.014')
ax.scatter(massIM_WO1, coreMassIM_WO1, facecolors = 'none', edgecolors = 'b', marker = 's', 
           s = 150, label = r'series Z = 0.001; f = 0.014')
ax.scatter(massSM_WO1, coreMassSM_WO1, facecolors = 'none', edgecolors = 'r', marker = 'o', 
           s = 150, label = r'series Z = 0.02; f = 0.014')

#ax.scatter(massLM_WO2, coreMassLM_WO2, c = 'g', marker = 'D', label = 'series LM;WO2')
#ax.scatter(massIM_WO2, coreMassIM_WO2, c = 'b', marker = 's', label = 'series IM;WO2')
#ax.scatter(massSM_WO2, coreMassSM_WO2, c = 'r', marker = 'o', label = 'series SM;WO2')

legend = ax.legend(loc = 'lower right', shadow = True, prop={'size':20})
# -------------------------------------------------------------



# Fitting curves + interpolated data
# -------------------------------------------------------------

#ax.plot(x_fit_LMWO1, y_fit_LMWO1, c = 'g', linestyle = '--')
#ax.scatter(newx_LMWNO, newy_LMWNO, facecolors = 'none', edgecolors = 'g', marker = 'D')

#ax.plot(x_fit_IMWO1, y_fit_IMWO1, c = 'b', linestyle = '--')
#ax.scatter(newx_IMWNO, newy_IMWNO, facecolors = 'none', edgecolors = 'b', marker = 's')

#ax.plot(x_fit_SMWO1, y_fit_SMWO1, c = 'r', linestyle = '--')
#ax.scatter(newx_SMWNO, newy_SMWNO, facecolors = 'none', edgecolors = 'r', marker = 'o')
# -------------------------------------------------------------




'''
# Final fate regions
# -------------------------------------------------------------
x_fill_between_12 = np.linspace(0.5, 1.3, 6)
y_fill_between_1 = poly_fit(x_fill_between_12, 0.18 , 2.18, 1.4)
y_fill_between_2 = poly_fit(x_fill_between_12, 0.2 , 2.35, 1.25)

x_fill_between_34 = np.linspace(1.3, 1.7, 5)
y_fill_between_3 = poly_fit(x_fill_between_34, 0.18 , 2.18, 1.4)
y_fill_between_4 = poly_fit(x_fill_between_34, 0.2 , 2.35, 1.25)

x_fill_between_56 = np.linspace(1.7, 2.0, 4)
y_fill_between_5 = poly_fit(x_fill_between_56, 0.18 , 2.18, 1.4)
y_fill_between_6 = poly_fit(x_fill_between_56, 0.2 , 2.35, 1.25)

x_fill_between_78 = np.linspace(2.0, 2.8, 9)
y_fill_between_7 = poly_fit(x_fill_between_78, 0.18 , 2.18, 1.4)
y_fill_between_8 = poly_fit(x_fill_between_78, 0.2 , 2.35, 1.25)

x_fill_between_910 = np.linspace(2.8, 3.6, 8)
y_fill_between_9 = poly_fit(x_fill_between_910, 0.18 , 2.18, 1.4)
y_fill_between_10 = poly_fit(x_fill_between_910, 0.2 , 2.35, 1.25)

ax.fill_between(x_fill_between_12, y_fill_between_1, y_fill_between_2, color = 'g', alpha = 0.1)
ax.fill_between(x_fill_between_34, y_fill_between_3, y_fill_between_4, color = 'y', alpha = 0.1)
ax.fill_between(x_fill_between_56, y_fill_between_5, y_fill_between_6, color = 'purple', alpha = 0.1)
ax.fill_between(x_fill_between_78, y_fill_between_7, y_fill_between_8, facecolor = 'none', hatch = 'X',
                edgecolor="purple", alpha = 0.3)
ax.fill_between(x_fill_between_910, y_fill_between_9, y_fill_between_10, color = 'r', alpha = 0.1)
'''



#plt.savefig('coreGrowth_WO1.pdf')

# revert changes to the configuration made by homogenize_plot:
#hp.revert_params()

In [ ]:
hp.homogenise_plot(fig_width = 10, fig_height = 10, columns = 2, fontsize = 20)


# For interpolated data
#newx_SMWO2, newy_SMWO2 = [], []
#newx_LMWO2, newy_LMWO2 = [], []
#newx_IMWO2, newy_IMWO2 = [], []


# Fitting parameters
# -------------------------------------------------------------
#popt, pcov = so.curve_fit(piecewise_linear, massSM_WO2[:16], coreMassSM_WO2[:16])
#print(popt)
#print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
#print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))
#print("c = %f +/- %f" % (popt[2], np.sqrt(pcov[2][2])))
#print("c = %f +/- %f" % (popt[3], np.sqrt(pcov[3][3])))
#x_fit_SMWO2 = np.linspace(0.8, 2.5, 18)
#y_fit_SMWO2 = piecewise_linear(x_fit_SMWO2, *popt)

#popt, pcov = so.curve_fit(piecewise_linear, massLM_WO2[:10], coreMassLM_WO2[:10])
#print(popt)
#print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
#print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))
#print("c = %f +/- %f" % (popt[2], np.sqrt(pcov[2][2])))
#x_fit_LMWO2 = np.linspace(0.8, 2.5, 18)
#y_fit_LMWO2 = piecewise_linear(x_fit_LMWO2,*popt)

#popt, pcov = so.curve_fit(piecewise_linear, massIM_WO2[:12], coreMassIM_WO2[:12])
#print(popt)
#print("a = %f +/- %f" % (popt[0], np.sqrt(pcov[0][0])))
#print("b = %f +/- %f" % (popt[1], np.sqrt(pcov[1][1])))
#print("c = %f +/- %f" % (popt[2], np.sqrt(pcov[2][2])))
#x_fit_IMWO2 = np.linspace(0.8, 2.5, 18)
#y_fit_IMWO2 = piecewise_linear(x_fit_IMWO2, *popt)


'''
# Interpolated data points
# -------------------------------------------------------------
for i in x_fit:
    found = False
    for j in massSM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_SMWO2.append(i)
        newy_SMWO2.append(poly_fit(i, 0.2, 2.32, 1.34))
        
for i in x_fit:
    found = False
    for j in massLM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_LMWO2.append(i)
        newy_LMWO2.append(poly_fit(i, 0.2, 2.145, 1.289))
        
for i in x_fit:
    found = False
    for j in massIM_WNO:
        
        if math.isclose(i,j, rel_tol = 0.001):
            #print(f'Value {i} is close to {j}')
            found = True
            break
    if not found:
        newx_IMWO2.append(i)
        newy_IMWO2.append(poly_fit(i, 0.2, 2.206, 1.32))
# -------------------------------------------------------------
'''    

# Plot
# -------------------------------------------------------------

fig, ax = plt.subplots()
ax.set_xlabel(r'Initial Mass [M$_{\odot}$]')
ax.set_ylabel(r'Final Core Mass [M$_{\odot}$]')

ax.set_xlim(0.5, 3.6)
ax.set_ylim(0.5, 1.9)
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))



ax.scatter(massLM_WO2, coreMassLM_WO2, facecolors = 'none', edgecolors = 'g', marker = 'd', 
           s= 150, label = r'series Z = 0.0001; f = 0.016')
ax.scatter(massIM_WO2, coreMassIM_WO2, facecolors = 'none', edgecolors = 'b', marker = 's', 
           s = 150, label = r'series Z = 0.001; f = 0.016')
ax.scatter(massSM_WO2, coreMassSM_WO2, facecolors = 'none', edgecolors = 'r', marker = 'o', 
           s = 150, label = r'series Z = 0.02; f = 0.016')

legend = ax.legend(loc = 'lower right', prop={'size': 20}, shadow = True)
# -------------------------------------------------------------


'''
# Fitting curves + interpolated data
# -------------------------------------------------------------

ax.plot(x_fit_LMWO2, y_fit_LMWO2, c = 'g', linestyle = '--')
#ax.scatter(newx_LMWNO, newy_LMWNO, facecolors = 'none', edgecolors = 'g', marker = 'D')

ax.plot(x_fit_IMWO2, y_fit_IMWO2, c = 'b', linestyle = '--')
#ax.scatter(newx_IMWNO, newy_IMWNO, facecolors = 'none', edgecolors = 'b', marker = 's')

ax.plot(x_fit_SMWO2, y_fit_SMWO2, c = 'r', linestyle = '--')
#ax.scatter(newx_SMWO2, newy_SMWO2, facecolors = 'none', edgecolors = 'r', marker = 'o')
# -------------------------------------------------------------





# Final fate regions
# -------------------------------------------------------------
x_fill_between_12 = np.linspace(0.5, 1.3, 6)
y_fill_between_1 = poly_fit(x_fill_between_12, 0.18 , 2.18, 1.4)
y_fill_between_2 = poly_fit(x_fill_between_12, 0.2 , 2.35, 1.25)

x_fill_between_34 = np.linspace(1.3, 1.7, 5)
y_fill_between_3 = poly_fit(x_fill_between_34, 0.18 , 2.18, 1.4)
y_fill_between_4 = poly_fit(x_fill_between_34, 0.2 , 2.35, 1.25)

x_fill_between_56 = np.linspace(1.7, 2.0, 4)
y_fill_between_5 = poly_fit(x_fill_between_56, 0.18 , 2.18, 1.4)
y_fill_between_6 = poly_fit(x_fill_between_56, 0.2 , 2.35, 1.25)

x_fill_between_78 = np.linspace(2.0, 2.8, 9)
y_fill_between_7 = poly_fit(x_fill_between_78, 0.18 , 2.18, 1.4)
y_fill_between_8 = poly_fit(x_fill_between_78, 0.2 , 2.35, 1.25)

x_fill_between_910 = np.linspace(2.8, 3.6, 8)
y_fill_between_9 = poly_fit(x_fill_between_910, 0.18 , 2.18, 1.4)
y_fill_between_10 = poly_fit(x_fill_between_910, 0.2 , 2.35, 1.25)

ax.fill_between(x_fill_between_12, y_fill_between_1, y_fill_between_2, color = 'g', alpha = 0.1)
ax.fill_between(x_fill_between_34, y_fill_between_3, y_fill_between_4, color = 'y', alpha = 0.1)
ax.fill_between(x_fill_between_56, y_fill_between_5, y_fill_between_6, color = 'purple', alpha = 0.1)
ax.fill_between(x_fill_between_78, y_fill_between_7, y_fill_between_8, facecolor = 'none', hatch = 'X',
                edgecolor="purple", alpha = 0.3)
ax.fill_between(x_fill_between_910, y_fill_between_9, y_fill_between_10, color = 'r', alpha = 0.1)

'''


#plt.savefig('coreGrowth_WO2.pdf')

# revert changes to the configuration made by homogenize_plot:
#hp.revert_params()

In [ ]:
hp.homogenise_plot(fig_width = 10, fig_height = 15, columns = 2, fontsize = 20)
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex='all', sharey = 'all')

ax1.scatter(massLM_WNO, coreMassLM_WNO, facecolors = 'none', edgecolors = 'g', marker = 'd',
            s = 150, label = r'series Z = 0.0001; f = 0.0')
ax1.scatter(massIM_WNO, coreMassIM_WNO, facecolors = 'none', edgecolors = 'b', marker = 's', 
            s = 150, label = r'series Z = 0.001; f = 0.0')
ax1.scatter(massSM_WNO, coreMassSM_WNO, marker = 'o', 
            s = 150, facecolors = 'none', edgecolors = 'r', label = r'series Z = 0.02; f = 0.0')

ax2.scatter(massLM_WO1, coreMassLM_WO1, facecolors = 'none', edgecolors = 'g', marker = 'd', 
            s = 150, label = r'series Z = 0.0001; f = 0.014')
ax2.scatter(massIM_WO1, coreMassIM_WO1, facecolors = 'none', edgecolors = 'b', marker = 's', 
            s = 150, label = r'series Z = 0.001; f = 0.014')
ax2.scatter(massSM_WO1, coreMassSM_WO1, facecolors = 'none', edgecolors = 'r', marker = 'o', 
            s = 150, label = r'series Z = 0.02; f = 0.014')

ax3.scatter(massLM_WO2, coreMassLM_WO2, facecolors = 'none', edgecolors = 'g', marker = 'd', 
            s = 150, label = r'series Z = 0.0001; f = 0.016')
ax3.scatter(massIM_WO2, coreMassIM_WO2, facecolors = 'none', edgecolors = 'b', marker = 's',
            s = 150, label = r'series Z = 0.001; f = 0.016')
ax3.scatter(massSM_WO2, coreMassSM_WO2, facecolors = 'none', edgecolors = 'r', marker = 'o',
            s = 150, label = r'series Z = 0.02; f = 0.016')

ax1.xaxis.set_major_locator(plt.MaxNLocator(10))
ax1.yaxis.set_major_locator(plt.MaxNLocator(10))

ax2.yaxis.set_major_locator(plt.MaxNLocator(10))
ax3.yaxis.set_major_locator(plt.MaxNLocator(10))



ax1.legend(loc = 'lower right', prop={'size':20}, shadow = True)
ax2.legend(loc = 'lower right', prop={'size':20}, shadow = True)
ax3.legend(loc = 'lower right', prop={'size':20}, shadow = True)

ax3.set_xlabel(r'Initial Mass [M$_{\odot}$]')

ax1.set_ylabel(r'Core Mass [M$_{\odot}$]')
ax2.set_ylabel(r'Core Mass [M$_{\odot}$]')
ax3.set_ylabel(r'Core Mass [M$_{\odot}$]')

#ax1.get_shared_x_axes().join(ax1, ax2, ax3)
#ax1.get_shared_y_axes().join(ax1, ax2, ax3)

#plt.savefig('coreGrowth_merge1.pdf')

In [ ]:
hp.homogenise_plot(fig_width = 10, fig_height = 15, columns = 2, fontsize = 20)
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex='all', sharey = 'all')

ax1.scatter(massLM_WNO, coreMassLM_WNO, facecolors = 'none', edgecolors = 'g', marker = 'd',
            s = 150, label = r'series Z = 0.0001; f = 0.0')
ax1.scatter(massLM_WO1, coreMassLM_WO1, facecolors = 'none', edgecolors = 'b', marker = 's', 
            s = 150, label = r'series Z = 0.0001; f = 0.014')
ax1.scatter(massLM_WO2, coreMassLM_WO2, marker = 'o', 
            s = 150, facecolors = 'none', edgecolors = 'r', label = r'series Z = 0.0001; f = 0.016')

ax2.scatter(massIM_WNO, coreMassIM_WNO, facecolors = 'none', edgecolors = 'g', marker = 'd', 
            s = 150, label = r'series Z = 0.001; f = 0.0')
ax2.scatter(massIM_WO1, coreMassIM_WO1, facecolors = 'none', edgecolors = 'b', marker = 's', 
            s = 150, label = r'series Z = 0.001; f = 0.014')
ax2.scatter(massIM_WO2, coreMassIM_WO2, facecolors = 'none', edgecolors = 'r', marker = 'o', 
            s = 150, label = r'series Z = 0.001; f = 0.016')

ax3.scatter(massSM_WNO, coreMassSM_WNO, facecolors = 'none', edgecolors = 'g', marker = 'd', 
            s = 150, label = r'series Z = 0.02; f = 0.00')
ax3.scatter(massSM_WO1, coreMassSM_WO1, facecolors = 'none', edgecolors = 'b', marker = 's',
            s = 150, label = r'series Z = 0.02; f = 0.014')
ax3.scatter(massSM_WO2, coreMassSM_WO2, facecolors = 'none', edgecolors = 'r', marker = 'o',
            s = 150, label = r'series Z = 0.02; f = 0.016')

ax1.xaxis.set_major_locator(plt.MaxNLocator(10))
ax1.yaxis.set_major_locator(plt.MaxNLocator(10))

ax2.yaxis.set_major_locator(plt.MaxNLocator(10))
ax3.yaxis.set_major_locator(plt.MaxNLocator(10))



ax1.legend(loc = 'lower right', prop={'size':20}, shadow = True)
ax2.legend(loc = 'lower right', prop={'size':20}, shadow = True)
ax3.legend(loc = 'lower right', prop={'size':20}, shadow = True)

ax3.set_xlabel(r'Initial Mass [M$_{\odot}$]')

ax1.set_ylabel(r'Core Mass [M$_{\odot}$]')
ax2.set_ylabel(r'Core Mass [M$_{\odot}$]')
ax3.set_ylabel(r'Core Mass [M$_{\odot}$]')

#plt.savefig('coreGrowth_merge2.pdf')

### Keep overshooting factor constant -> core growth

In [ ]:
# Plot data with no overshooting


#popt_k, pcov_k = so.curve_fit(poly_fit, massLM_WNO, coreMassLM_WNO)
#popt_b, pcov_b = so.curve_fit(poly_fit, massIM_WNO, coreMassIM_WNO)
#popt_r, pcov_r = so.curve_fit(poly_fit, massSM_WNO, coreMassSM_WNO)

#print(popt_k, pcov_k)
#print(popt_b, pcov_b)
#print(popt_r, pcov_r)



#print("a = %f +/- %f" % (popt_k[0], np.sqrt(pcov_k[0][0])))
#print("b = %f +/- %f" % (popt_k[1], np.sqrt(pcov_k[1][1])))
#print("c = %f +/- %f" % (popt_k[2], np.sqrt(pcov_k[2][2])))
#print("a = %f +/- %f" % (popt_b[0], np.sqrt(pcov_b[0][0])))
#print("b = %f +/- %f" % (popt_b[1], np.sqrt(pcov_b[1][1])))
#print("c = %f +/- %f" % (popt_b[2], np.sqrt(pcov_b[2][2])))
#print("a = %f +/- %f" % (popt_r[0], np.sqrt(pcov_r[0][0])))
#print("b = %f +/- %f" % (popt_r[1], np.sqrt(pcov_r[1][1])))
#print("c = %f +/- %f" % (popt_r[2], np.sqrt(pcov_r[2][2])))


# Data for fitting curve
#x_fit = np.linspace(0.8, 3.5, 14)
#y_fit_k = poly_fit(x_fit, *(popt_k))
#y_fit_b = poly_fit(x_fit, *(popt_b))
#y_fit_r = poly_fit(x_fit, *(popt_r))



fig, ax = plt.subplots()
#ax.set_title('WNO')
ax.set_xlabel(r'Initial mass [M$_{\odot}$]')
ax.set_ylabel(r'Core mass [M$_{\odot}$]')

ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))
ax.set_xlim(0.5, 3.7)


# Plot
ax.scatter(massLM_WNO, coreMassLM_WNO, c = 'g', s=50, 
           label = 'LM;WNO', marker = 'D')

ax.scatter(xk_new, yk_new, s = 50, marker = 'D', edgecolors = 'g', facecolors = 'none')


ax.scatter(massIM_WNO, coreMassIM_WNO, c = 'b', s=50, 
           label = 'IM;WNO', marker = 's')

ax.scatter(xb_new, yb_new, s = 50, marker = 's', edgecolors = 'b', facecolors = 'none')


ax.scatter(massSM_WNO, coreMassSM_WNO, c = 'r', s=50, 
           label = 'SM;WNO', marker = 'o')

ax.scatter(xr_new, yr_new, s = 50, marker = 'o', edgecolors = 'r', facecolors = 'none')


# Fit
ax.plot(x_fit, y_fit_k, c = 'g', linestyle = '--')
ax.plot(x_fit, y_fit_b, c = 'b', linestyle = '--')
ax.plot(x_fit, y_fit_r, c = 'r', linestyle = '--')


ax.fill([1.4, 3.8, 3.8, 1.4], [1.365, 1.365, 1.45, 1.45], 'purple', alpha = 0.05)
ax.fill([0.4, 1.4, 1.4, 0.4], [1.365, 1.365, 1.45, 1.45], hatch="X", color="purple", alpha = 0.1 )

legend = ax.legend(prop={'size' : 13}, shadow = True)
#textstr = '\n'.join(('Fit function:', r'$M_c \approx a \left(M_{i} - b \right)^{3} + c $'))


#props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.09)
#plt.text(0.55, 1.48, textstr, fontsize = 14, bbox = props)

#plt.savefig(f'{plot_out}/Cores/core_growth_WNO.pdf', bbox_inches = 'tight', dpi = 300)


In [ ]:
# Plot data with f = 0.0140

fig, ax = plt.subplots(figsize = (13,9))
#ax.set_title('WO1')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Core mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))

ax.scatter(massLM_WO1, coreMassLM_WO1, c = 'g', s=50, label = 'LM;WO1', marker = 'D')
ax.scatter(massIM_WO1, coreMassIM_WO1, c = 'b', s=50, label = 'IM;WO1', marker = 's')
ax.scatter(massSM_WO1, coreMassSM_WO1, c = 'r', s=50, label = 'SM;WO1', marker = 'o')


ax.set_xlim(0.5, 3.7)

ax.fill([0.4, 3.8, 3.8, 0.4], [1.365, 1.365, 1.45, 1.45], 'purple', alpha = 0.05)
ax.fill([0.4, 1.4, 1.4, 0.4], [1.365, 1.365, 1.45, 1.45], hatch="X", color="purple", alpha = 0.1 )

legend = ax.legend(loc = 'upper left', prop={'size' : 13}, shadow = True)

#plt.savefig(f'{plot_out}/Cores/core_growth_WO1.pdf', bbox_inches = 'tight', dpi = 300)

In [ ]:
# Plot data with f = 0.0160

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('WO2')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Core mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))

ax.scatter(massLM_WO2, coreMassLM_WO2, c = 'g', s=50, label = 'LM;WO2', marker = 'D')
ax.scatter(massIM_WO2, coreMassIM_WO2, c = 'b', s=50, label = 'IM;WO2', marker = 's')
ax.scatter(massSM_WO2, coreMassSM_WO2, c = 'r', s=50, label = 'SM;WO2', marker = 'o')

ax.set_xlim(0.5, 3.7)

ax.fill([0.4, 3.8, 3.8, 0.4], [1.365, 1.365, 1.45, 1.45], 'purple', alpha = 0.05)
ax.fill([0.4, 1.4, 1.4, 0.4], [1.365, 1.365, 1.45, 1.45], hatch="X", color="purple", alpha = 0.1 )

legend = ax.legend(loc = 'upper left', prop={'size' : 13}, shadow = True)

#plt.savefig(f'{plot_out}/Cores/core_growth_WO2.pdf', bbox_inches = 'tight', dpi = 300)

### Keep initial metallicity constant -> core growth

In [ ]:
# Plot data with low metallicity, Z = 0.0001

popt_k, pcov_k = so.curve_fit(poly_fit, massLM_WNO, coreMassLM_WNO)

x_fit = np.linspace(0.8, 3.5, 14)
y_fit_k = poly_fit(x_fit, *(popt_k))

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('LM')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Core mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))

ax.scatter(massLM_WNO, coreMassLM_WNO, marker = 'D', c = 'g', s=50, label = 'LM;WNO')
ax.scatter(massLM_WO1, coreMassLM_WO1, marker = 's', c = 'b', s=50, label = 'LM;WO1')
ax.scatter(massLM_WO2, coreMassLM_WO2, marker = 'o', c = 'r', s=50, label = 'LM;WO2')

ax.plot(x_fit, y_fit_k, c = 'g', linestyle = '--')
ax.scatter(xk_new, yk_new, s = 50, marker = 'D', edgecolors = 'g', facecolors = 'none')

ax.set_xlim(0.5, 3.7)

ax.fill([0.4, 3.8, 3.8, 0.4], [1.365, 1.365, 1.45, 1.45], 'purple', alpha = 0.05)
ax.fill([0.4, 1.4, 1.4, 0.4], [1.365, 1.365, 1.45, 1.45], hatch="X", color="purple", alpha = 0.1 )

legend = ax.legend(loc = 'upper left', prop={'size' : 13})

#plt.savefig(f'{plot_out}/Cores/core_growth_LM.pdf', bbox_inches = 'tight', dpi = 300)

In [ ]:
# Plot data with intermediate metallicity, Z = 0.0010

popt_k, pcov_k = so.curve_fit(poly_fit, massIM_WNO, coreMassIM_WNO)

x_fit = np.linspace(0.8, 3.5, 14)
y_fit_k = poly_fit(x_fit, *(popt_k))

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('IM')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Core mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))

ax.scatter(massIM_WNO, coreMassIM_WNO, marker = 'D', c = 'g', s=50, label = 'IM;WNO')
ax.scatter(massIM_WO1, coreMassIM_WO1, marker = 's', c = 'b', s=50, label = 'IM;WO1')
ax.scatter(massIM_WO2, coreMassIM_WO2, marker = 'o', c = 'r', s=50, label = 'IM;WO2')

ax.plot(x_fit, y_fit_k, c = 'g', linestyle = '--')
ax.scatter(xb_new, yb_new, s = 50, marker = 'D', edgecolors = 'g', facecolors = 'none')

ax.set_xlim(0.5, 3.7)

ax.fill([0.4, 3.8, 3.8, 0.4], [1.365, 1.365, 1.45, 1.45], 'purple', alpha = 0.05)
ax.fill([0.4, 1.4, 1.4, 0.4], [1.365, 1.365, 1.45, 1.45], hatch="X", color="purple", alpha = 0.1 )

legend = ax.legend(loc = 'upper left', prop={'size' : 13})

#plt.savefig(f'{plot_out}/Cores/core_growth_IM.pdf', bbox_inches = 'tight', dpi = 300)

In [ ]:
print(massSM_WO1[:13])

In [ ]:
# Plot data with solar metallicity, Z = 0.0200

popt_SMWNO, pcov_SMWNO = so.curve_fit(piecewise_linear, massSM_WNO[:17], coreMassSM_WNO[:17])

x_fit_SMWNO = np.linspace(0.8, 2.7, 14)
y_fit_SMWNO = piecewise_linear(x_fit_SMWNO, *(popt_SMWNO))

popt_SMWO2, pcov_SMWO2 = so.curve_fit(piecewise_linear, massSM_WO2[:16], coreMassSM_WO2[:16])

x_fit_SMWO2 = np.linspace(0.8, 2.3, 14)
y_fit_SMWO2 = piecewise_linear(x_fit_SMWO2, *(popt_SMWO2))

popt_SMWO1, pcov_SMWO1 = so.curve_fit(piecewise_linear, massSM_WO2[:12], coreMassSM_WO2[:12])

x_fit_SMWO1 = np.linspace(0.8, 2.3, 14)
y_fit_SMWO1 = piecewise_linear(x_fit_SMWO1, *(popt_SMWO1))

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('SM')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Core mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.yaxis.set_major_locator(plt.MaxNLocator(10))

ax.scatter(massSM_WNO, coreMassSM_WNO, marker = 'D', c = 'g', s=50, label = 'SM;WNO')
ax.scatter(massSM_WO1, coreMassSM_WO1, marker = 's', c = 'b', s=50, label = 'SM;WO1')
ax.scatter(massSM_WO2, coreMassSM_WO2, marker = 'o', c = 'r', s=50, label = 'SM;WO2')

ax.plot(x_fit_SMWNO, y_fit_SMWNO, c = 'g', linestyle = '--')
ax.plot(x_fit_SMWO1, y_fit_SMWO1, c = 'b', linestyle = '--')
ax.plot(x_fit_SMWO2, y_fit_SMWO2, c = 'r', linestyle = '--')
#ax.scatter(xr_new, yr_new, s = 50, marker = 'D', edgecolors = 'g', facecolors = 'none')

ax.set_xlim(0.5, 3.7)
ax.set_ylim(0.5, 1.8)

legend = ax.legend(loc = 'upper left', prop={'size' : 13})


# Final fate regions
# -------------------------------------------------------------
x_fill_between_12 = np.linspace(0.5, 1.3, 6)
y_fill_between_1 = poly_fit(x_fill_between_12, 0.18 , 2.18, 1.4)
y_fill_between_2 = poly_fit(x_fill_between_12, 0.2 , 2.35, 1.25)

x_fill_between_34 = np.linspace(1.3, 1.7, 5)
y_fill_between_3 = poly_fit(x_fill_between_34, 0.18 , 2.18, 1.4)
y_fill_between_4 = poly_fit(x_fill_between_34, 0.2 , 2.35, 1.25)

x_fill_between_56 = np.linspace(1.7, 2.0, 4)
y_fill_between_5 = poly_fit(x_fill_between_56, 0.18 , 2.18, 1.4)
y_fill_between_6 = poly_fit(x_fill_between_56, 0.2 , 2.35, 1.25)

x_fill_between_78 = np.linspace(2.0, 2.8, 9)
y_fill_between_7 = poly_fit(x_fill_between_78, 0.18 , 2.18, 1.4)
y_fill_between_8 = poly_fit(x_fill_between_78, 0.2 , 2.35, 1.25)

x_fill_between_910 = np.linspace(2.8, 3.7, 8)
y_fill_between_9 = poly_fit(x_fill_between_910, 0.18 , 2.18, 1.4)
y_fill_between_10 = poly_fit(x_fill_between_910, 0.2 , 2.35, 1.25)

ax.fill_between(x_fill_between_12, y_fill_between_1, y_fill_between_2, color = 'g', alpha = 0.1)
ax.fill_between(x_fill_between_34, y_fill_between_3, y_fill_between_4, color = 'y', alpha = 0.1)
ax.fill_between(x_fill_between_56, y_fill_between_5, y_fill_between_6, color = 'purple', alpha = 0.1)
ax.fill_between(x_fill_between_78, y_fill_between_7, y_fill_between_8, facecolor = 'none', hatch = 'X',
                edgecolor="purple", alpha = 0.3)
ax.fill_between(x_fill_between_910, y_fill_between_9, y_fill_between_10, color = 'r', alpha = 0.1)

#plt.savefig(f'{plot_out}/Cores/core_growth_SM.pdf', bbox_inches = 'tight', dpi = 300)

### Keep overshooting factor constant -> envelope mass retain

In [ ]:
# Plot data with no overshooting

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('WNO')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Envelope mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(13))
ax.yaxis.set_major_locator(plt.MaxNLocator(13))

ax.scatter(massLM_WNO, envMassLM_WNO, facecolors = 'none', edgecolors = 'k', s=50, label = 'LM;WNO')
ax.scatter(massIM_WNO, envMassIM_WNO, facecolors = 'none', edgecolors = 'b', s=50, label = 'IM;WNO')
ax.scatter(massSM_WNO, envMassSM_WNO, facecolors = 'none', edgecolors = 'r', s=50, label = 'SM;WNO')

legend = ax.legend(loc = 'upper left', prop={'size' : 13})

#plt.savefig(f'{plot_out}/Core_growth/envelope_mass_WNO.pdf', bbox_inches = 'tight', dpi = 300)

In [ ]:
# Plot data with f = 0.0140

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('WO1')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Envelope mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(13))
ax.yaxis.set_major_locator(plt.MaxNLocator(13))

ax.scatter(massLM_WO1, envMassLM_WO1, facecolors = 'none', edgecolors = 'k', s=50, label = 'LM;WO1')
ax.scatter(massIM_WO1, envMassIM_WO1, facecolors = 'none', edgecolors = 'b', s=50, label = 'IM;WO1')
ax.scatter(massSM_WO1, envMassSM_WO1, facecolors = 'none', edgecolors = 'r', s=50, label = 'SM;WO1')

legend = ax.legend(loc = 'upper left', prop={'size' : 13})

#plt.savefig(f'{plot_out}/Core_growth/envelope_mass_WO1.pdf', bbox_inches = 'tight', dpi = 300)

In [ ]:
# Plot data with f = 0.0160

fig, ax = plt.subplots(figsize = (13,9))
#plt.figure(figsize = (13,9))
#ax.set_title('WO2')

ax.set_xlabel(r'Initial mass [M$_{\odot}$]', fontsize = 15)
ax.set_ylabel(r'Envelope mass [M$_{\odot}$]', fontsize = 15)

ax.xaxis.set_major_locator(plt.MaxNLocator(13))
ax.yaxis.set_major_locator(plt.MaxNLocator(13))

ax.scatter(massLM_WO2, envMassLM_WO2, facecolors = 'none', edgecolors = 'k', s=50, label = 'LM;WO2')
ax.scatter(massIM_WO2, envMassIM_WO2, facecolors = 'none', edgecolors = 'b', s=50, label = 'IM;WO2')
ax.scatter(massSM_WO2, envMassSM_WO2, facecolors = 'none', edgecolors = 'r', s=50, label = 'SM;WO2')

legend = ax.legend(loc = 'upper left', prop={'size' : 13})

#plt.savefig(f'{plot_out}/Core_growth/envelope_mass_WO2.pdf', bbox_inches = 'tight', dpi = 300)

## Step 4: Plot central density vs central temperature
---

In order to plot the Rho_c vs T_c diagram, we're going to use once again the **[mesa_star_class]** and its plotRhoT() method.

In [ ]:
for path in glob.glob(f'{full_grid}/*'):
    
    string_input = path.split('/')[-1]
    logs_path = os.path.join(path, 'LOGS')
    
    if ms.name_is_valid(string_input):

        try:
            os.system(f'cd {logs_path}')

            # Find the last saved profile from the profiles.index 
            prof_num = ms.find_profile_number(logs_path, num = -1)

            
            # Define a MESA_STAR object
            star = ms.from_string(string_input,
                                 history_path = logs_path,
                                 profile_path = logs_path,
                                 profile_number = prof_num)
            
            figureName = f'Rho_vs_T_{string_input}.pdf'

            star.plotRhoT(saveFigure=True, figureName=figureName, plot_output_dir=f'{plot_out}/RhoT_new/new')
            
            
        except Exception as e:
            print(f'Something went wrong when trying to load the data from {path}')
            print(e)
    else:
        print(f'Name is not valid for {path}')

## Step 5: Plot abundances
---

Here we're going to use the **[mesaPlot]** module.

For more info see Robert Farmer's [git repo](https://github.com/rjfarmer/mesaplot/tree/b369d840b96b1f385ff4decac74bf141ed80783b)

In [ ]:
for path in glob.glob(f'{full_grid}/*'):
    
    string_input = path.split('/')[-1]
    logs_path = os.path.join(path, 'LOGS')
    
    if ms.name_is_valid(string_input):
    
        try:

            abun_plot_out = os.path.join(plot_out, f'Abundances/{string_input}')
            os.system(f'mkdir {abun_plot_out}')

            m = mp.MESA()
            m.log_fold = logs_path

            m.loadHistory()
            m.loadProfile(num = -1)
            p = mp.plot()

            # Create three different plots regarding abundances
            p.plotAbun(m, show = False)
            plt.savefig(f'{abun_plot_out}/final_abundance_profile.pdf', bbox_inches = 'tight', dpi = 300)
            plt.close('all')

            p.plotAbunPAndN(m, show = False)
            plt.savefig(f'{abun_plot_out}/AbunPandN.pdf', bbox_inches = 'tight', dpi = 300)
            plt.close('all')

            p.plotAbunHist(m, show = False)
            plt.savefig(f'{abun_plot_out}/AbunHist.pdf', bbox_inches = 'tight', dpi = 300)
            plt.close('all')

        except:
            print(f'Something went wrong when trying to plot abundance profiles for model {string_input}')
            
    else:
        print(f'Name is not valid for {path}')

## Step 6: Kippenhahn plots
---

Once again, we're using the **{mesaPlot}** module for plotting Kippenhahn diagrams.

In [ ]:
for path in glob.glob(f'{full_grid}/*'):
    
    string_input = path.split('/')[-1]
    logs_path = os.path.join(path, 'LOGS')
    
    if ms.name_is_valid(string_input):
    
        try:

            kipp_plot_out = os.path.join(plot_out, f'Kipp/{string_input}')
            os.system(f'mkdir {kipp_plot_out}')

            m = mp.MESA()
            m.log_fold = logs_path

            m.loadHistory()
            m.loadProfile(num = -1)
            p = mp.plot()

            # Create three different Kippenhahn plots

            p.plotKip(m,show_mass_loc=True, show = False)
            plt.savefig(f'{kipp_plot_out}/Kipp_1.pdf', bbox_inches = 'tight', dpi = 300)
            plt.close('all')

            #p.plotKip2(m, show = False)
            #plt.savefig(f'{kipp_plot_out}/Kipp_2.pdf', bbox_inches = 'tight', dpi = 300)
            #plt.close('all')

            #p.plotKip3(m,plot_type='profile',xaxis='model_number',yaxis='mass',zaxis='logRho',mod_min=1,mod_max=3000, show = False)
            #plt.savefig(f'{kipp_plot_out}/Kipp_3.pdf', bbox_inches = 'tight', dpi = 300)
            #plt.close('all')

        except Exception as e:
            print(f'Something went wrong when trying to create Kippenhahn plots for model {string_input}')
            print(e)
            
                
    else:
        print(f'Name is not valid for {path}')

### Compare final fates

In [ ]:

m_path = ['/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data/0.8000_0.0001_0.0000/LOGS',
          '/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data/1.4000_0.0010_0.0160/LOGS',
          '/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data/1.8000_0.0200_0.0140/LOGS',
          '/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data/2.5000_0.0200_0.0000/LOGS',
          '/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data/3.2000_0.0001_0.0140/LOGS']


label = [r'0.8M$_{\odot}, LM;WNO$',
         r'1.4M$_{\odot}, IM;WO2$',
         r'1.8M$_{\odot}, SM;WO1$',
         r'2.5M$_{\odot}, SM;WNO$',
         r'3.2M$_{\odot}, LM;WO1$']

colors = iter(['blue', 'green', 'magenta', 'orange', 'brown'])

for idx, path in enumerate(m_path):
    
    info = path.split('/')
    
    star = ms.from_string(info[-2], history_path = path)
    
    
    star.plotRhoT(ylim = [7.8, 10.0], color = next(colors), overplot=True, saveFigure=True, plot_output_dir='/users/schanlar/Desktop')

In [ ]:
n_path = ['/vol/aibn1107/data2/schanlar/HeCoresCondor/full_data/2.5000_0.0200_0.0000/LOGS',
          '/vol/aibn1107/data2/schanlar/HeCoresCondor/analysis']

history_names = ['history', 'history_ecsn']
colors = iter(['blue', 'green', 'magenta', 'orange', 'brown'])

for idx, path in enumerate(n_path):
    
    star = ms(mass='2.5000', metallicity='0.0200', overshooting='0.0000', history_name=history_names[idx],
             history_path = path)
    
    star.plotRhoT(ylim=[7.8, 10.0], color = next(colors), overplot=True, saveFigure=True, 
                  plot_output_dir='/users/schanlar/Desktop', figureName='RhoT_suppress.pdf')

### Parameter space grid and final fates

In [ ]:
mass_grid, metallicity_grid, overshooting_grid = [],[],[]
core_grid, envelope_grid = [], []
final_fates, explosions = [], []
termination_codes= []

with open('fullCoreGrowthData.csv', 'r') as file:
    next(file)
    reader = csv.reader(file, delimiter = ',')
    
    for row in reader:
        
        mass_grid.append(float(row[0]))
        metallicity_grid.append(float(row[1]))
        overshooting_grid.append(float(row[2]))
        
        core_grid.append(float(row[3]))
        envelope_grid.append(float(row[4]))
        
        final_fates.append(row[5])
        explosions.append(row[6])
        
        termination_codes.append(row[7])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
hp.homogenise_plot(columns=2, fig_width=15, fig_height = 13, fontsize = 17)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


colors = ['blue', 'green', 'magenta', 'orange', 'red']
shapes = ['d', '*', 'o', '.']
sizes = [300, 1000]

yticks = [r'$0.0001$', r'$0.001$', r'$0.02$']
zticks = [r'$0.0$', r'$0.014$', r'$0.016$']

for m,z,f,fate,e in zip(mass_grid, metallicity_grid, overshooting_grid, final_fates, explosions):
    
    if z == 0.0001:
        z = 0.0
        marker = shapes[3]
    elif z == 0.001:
        z = 1.0
        marker = shapes[3]
    elif z == 0.02:
        z = 2.0
        marker = shapes[3]
    else:
        raise ValueError('Something went wrong with metallicities!')
        
    if f == 0.0:
        f = 0.0
    elif f == 0.014:
        f = 1.0
    elif f == 0.016:
        f = 2.0
    else:
        raise ValueError('Something went wrong with overshooting factor!')
        
        
    if fate == 'COWD' or fate == 'COWD*' or fate == 'COWD**':
        color = colors[0]
    elif fate == 'CONeWD' or fate == 'CONeWD*' or fate == 'CONeWD**' or fate == 'CONewD':
        color = colors[1]
    elif fate == 'ONeWD' or fate == 'ONeWD*' or fate == 'ONeWD**':
        color = colors[2]
    elif fate == 'ONeSiWD' or fate == 'ONeSiWD*' or fate == 'ONeSiWD**':
        color = colors[3]
    elif fate == 'SiCORE' or fate == 'SiCORE*' or fate == 'SiCORE**':
        color = colors[4]
    else:
        print(f'I did not find the fate for {fate}!')
        
    if e == 'NO':
        size = sizes[0]
    elif e == 'YES':
        size = sizes[1]
    else:
        print('I did not find the explosion!')
        
        
    ax.xaxis.set_major_locator(plt.MaxNLocator(15))
    ax.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax.zaxis.set_major_locator(plt.MaxNLocator(3))
    
    ax.view_init(azim=-25) # set viewing angle to -30deg
    ax.set_yticklabels(yticks)
    ax.set_zticklabels(zticks, rotation=90)
    ax.scatter(m, z, f, color=color, marker=marker, s=size, alpha = 0.3)

ax.set_xlabel(r'Initial Mass [M$_{\odot}$]')
ax.set_ylabel(r'Metallicity ($Z$)')
ax.set_zlabel(r'Overshooting (f)')

#plt.savefig('/users/schanlar/Desktop/parameterSpace3D.pdf', bbox_inches = 'tight', dpi=300)

In [ ]:
hp.homogenise_plot(columns=2, fig_width=25, fig_height = 23, fontsize = 25)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


colors = ['navy', 'dodgerblue', 'green', 'magenta', 'red']
shapes = ['d', '*', 'o', '.']
sizes = [300, 1000]

yticks = [r'$0.0001$', r'$0.001$', r'$0.02$']
zticks = [r'$0.0$', r'$0.014$', r'$0.016$']

for m,z,f,core,envelope,fate,e in zip(mass_grid, metallicity_grid, overshooting_grid, 
                                      core_grid, envelope_grid, final_fates, explosions):
    
    if z == 0.0001:
        z = 0.0
        marker = shapes[3]
    elif z == 0.001:
        z = 1.0
        marker = shapes[3]
    elif z == 0.02:
        z = 2.0
        marker = shapes[3]
    else:
        raise ValueError('Something went wrong with metallicities!')
        
    if f == 0.0:
        f = 0.0
    elif f == 0.014:
        f = 1.0
    elif f == 0.016:
        f = 2.0
    else:
        raise ValueError('Something went wrong with overshooting factor!')
        
       
    if fate == 'COWD' or fate == 'COWD*' or fate == 'COWD**':
        color = colors[0]
    elif fate == 'CONeWD' or fate == 'CONeWD*' or fate == 'CONeWD**' or fate == 'CONewD':
        color = colors[1]
    elif fate == 'ONeWD' or fate == 'ONeWD*' or fate == 'ONeWD**':
        color = colors[2]
    elif fate == 'ONeSiWD' or fate == 'ONeSiWD*' or fate == 'ONeSiWD**':
        color = colors[3]
    elif fate == 'SiCORE' or fate == 'SiCORE*' or fate == 'SiCORE**':
        color = colors[4]
    else:
        print(f'I did not find the fate for {fate}!')
        
    #if e == 'NO':
    #    size = sizes[0]
    #elif e == 'YES':
    #    size = sizes[1]
    #else:
    #    print('I did not find the explosion!')
    
    
    if envelope <= 0.3 and math.isclose(core, 1.35, rel_tol = 0.02):
        size = sizes[1]
    else:
        size = sizes[0]
        
    
        
        
    ax.xaxis.set_major_locator(plt.MaxNLocator(15))
    ax.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax.zaxis.set_major_locator(plt.MaxNLocator(3))
    ax.tick_params(axis = 'x', which = 'major')
    
    ax.view_init(azim=-25) # set viewing angle to -30deg
    ax.set_yticklabels(yticks)
    ax.set_zticklabels(zticks, rotation=90)
    ax.scatter(m, z, f, color=color, marker=marker, s=size, alpha = 0.3)

ax.set_xlabel(r'Initial Mass [M$_{\odot}$]', labelpad=30)
ax.set_ylabel(r'Metallicity ($Z$)', labelpad=30)
ax.set_zlabel(r'Overshooting (f)', labelpad=30)

#plt.savefig('/users/schanlar/Desktop/parameterSpace3D.pdf')

### Extract top, middle, and bottom layer of the 3D parameter space

In [ ]:
hp.homogenise_plot(fig_width = 15, fig_height = 15, columns = 2, fontsize=20)

fig, ax = plt.subplots()

for m,z,f,core,envelope,fate,e in zip(mass_grid, metallicity_grid, overshooting_grid, 
                                      core_grid, envelope_grid, final_fates, explosions):
    
    if z == 0.0001:
        z = 0.0
        marker = shapes[3]
    elif z == 0.001:
        z = 1.0
        marker = shapes[3]
    elif z == 0.02:
        z = 2.0
        marker = shapes[3]
    else:
        raise ValueError('Something went wrong with metallicities!')
        
    if f == 0.0:
        f = 0.0
    elif f == 0.014:
        f = 1.0
    elif f == 0.016:
        f = 2.0
    else:
        raise ValueError('Something went wrong with overshooting factor!')
        
       
    if fate == 'COWD' or fate == 'COWD*' or fate == 'COWD**':
        color = colors[0]
    elif fate == 'CONeWD' or fate == 'CONeWD*' or fate == 'CONeWD**' or fate == 'CONewD':
        color = colors[1]
    elif fate == 'ONeWD' or fate == 'ONeWD*' or fate == 'ONeWD**':
        color = colors[2]
    elif fate == 'ONeSiWD' or fate == 'ONeSiWD*' or fate == 'ONeSiWD**':
        color = colors[3]
    elif fate == 'SiCORE' or fate == 'SiCORE*' or fate == 'SiCORE**':
        color = colors[4]
    else:
        print(f'I did not find the fate for {fate}!')
        
    #if e == 'NO':
    #    size = sizes[0]
    #elif e == 'YES':
    #    size = sizes[1]
    #else:
    #    print('I did not find the explosion!')
    
    
    if envelope <= 0.3 and math.isclose(core, 1.35, rel_tol = 0.02):
        size = sizes[1]
    else:
        size = sizes[0]
    
    # Change this to 0.0, 1.0, or 2.0 for the
    # bottom, middle, and top layer respectively
    if f == 1.0:
        
        ax.scatter(m, z, color=color, marker=marker, s=size, alpha = 0.3)
        
        
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
ax.yaxis.set_major_locator(plt.MaxNLocator(1))
ax.set_yticklabels(yticks)
ax.set_xlabel(r'Initial Mass [M$_{\odot}$]')
ax.set_ylabel(r'Metallicity ($Z$)')


#plt.savefig('/users/schanlar/Desktop/parameterSpaceWO1.pdf', bbox_inches = 'tight', dpi=300)

In [ ]:
x_coordinate = np.linspace(0, 27, 28)
y1_coordinate = np.linspace(0, 8, 9)

X,Y = np.meshgrid(x_coordinate, y1_coordinate)

In [ ]:
def f(X,Y):
    
    m1,z1,f1,c1,e1,r1 = [],[],[],[],[],[]
    Z = np.zeros((9, 28))
    
    # Read data from file
    with open('fullCoreGrowthData.csv', 'r') as file:
        next(file)
        reader = csv.reader(file)
        
        for row in reader:
            m1.append(row[0])
            z1.append(row[1])
            f1.append(row[2])
            c1.append(float(row[3]))
            e1.append(float(row[4]))
            r1.append(row[5])
    
    
    
    for i,j,k in list(zip(X,Y,Z)):
        #for l,m,n in zip(i,j,k):
        
        for idx, value in enumerate(j):
    
            if math.isclose(value,0.0):
                metallicity = '0.0001'
                overshooting = '0.0000'

            elif math.isclose(value,1.0):
                metallicity = '0.0001'
                overshooting = '0.0140'

            elif math.isclose(value,2.0):
                metallicity = '0.0001'
                overshooting = '0.0160'

            elif math.isclose(value,3.0):
                metallicity = '0.0010'
                overshooting = '0.0000'

            elif math.isclose(value,4.0):
                metallicity = '0.0010'
                overshooting = '0.0140'

            elif math.isclose(value,5.0):
                metallicity = '0.0010'
                overshooting = '0.0160'

            elif math.isclose(value,6.0):
                metallicity = '0.0200'
                overshooting = '0.0000'

            elif math.isclose(value,7.0):
                metallicity = '0.0200'
                overshooting = '0.0140'

            elif math.isclose(value,8.0):
                metallicity = '0.0200'
                overshooting = '0.0160'


            if math.isclose(i[idx], 0.0):
                mass = '0.8000'

            elif math.isclose(i[idx], 1.0):
                mass = '0.9000'

            elif math.isclose(i[idx], 2.0):
                mass = '1.0000'

            elif math.isclose(i[idx], 3.0):
                mass = '1.1000'

            elif math.isclose(i[idx], 4.0):
                mass = '1.2000'

            elif math.isclose(i[idx], 5.0):
                mass = '1.3000'

            elif math.isclose(i[idx], 6.0):
                mass = '1.4000'

            elif math.isclose(i[idx], 7.0):
                mass = '1.5000'

            elif math.isclose(i[idx], 8.0):
                mass = '1.6000'

            elif math.isclose(i[idx], 9.0):
                mass = '1.7000'

            elif math.isclose(i[idx], 10.0):
                mass = '1.8000'

            elif math.isclose(i[idx], 11.0):
                mass = '1.9000'

            elif math.isclose(i[idx], 12.0):
                mass = '2.0000'

            elif math.isclose(i[idx], 13.0):
                mass = '2.1000'

            elif math.isclose(i[idx], 14.0):
                mass = '2.2000'

            elif math.isclose(i[idx], 15.0):
                mass = '2.3000'

            elif math.isclose(i[idx], 16.0):
                mass = '2.4000'

            elif math.isclose(i[idx], 17.0):
                mass = '2.5000'

            elif math.isclose(i[idx], 18.0):
                mass = '2.6000'

            elif math.isclose(i[idx], 19.0):
                mass = '2.7000'

            elif math.isclose(i[idx], 20.0):
                mass = '2.8000'

            elif math.isclose(i[idx], 21.0):
                mass = '2.9000'

            elif math.isclose(i[idx], 22.0):
                mass = '3.0000'

            elif math.isclose(i[idx], 23.0):
                mass = '3.1000'

            elif math.isclose(i[idx], 24.0):
                mass = '3.2000'

            elif math.isclose(i[idx], 25.0):
                mass = '3.3000'

            elif math.isclose(i[idx], 26.0):
                mass = '3.4000'

            elif math.isclose(i[idx], 27.0):
                mass = '3.5000'
        
        
    
            """
            for a1,a2,a3,a4,a5,a6 in zip(m1,z1,f1,c1,e1,r1):

                if a1 == mass and a2 == metallicity and a3 == overshooting:
                    
                    '''

                    if a5 <= 0.3 and math.isclose(a4,1.35, rel_tol=0.02):
                        k[idx] = 6

                    elif a6 == 'COWD' or a6 == 'COWD*' or a6 == 'COWD**':
                        k[idx] = 1

                    elif a6 == 'CONeWD' or a6 == 'CONeWD*' or a6 == 'CONeWD**' or a6 == 'CONewD':
                        k[idx] = 2

                    elif a6 == 'ONeWD' or a6 == 'ONeWD*' or a6 == 'ONeWD**':
                        k[idx] = 3

                    elif a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**':
                        k[idx] = 4

                    elif a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**':
                        k[idx] = 5
                        
                    '''
                    
                    if a5 <= 0.3:
                        if math.isclose(a4, 1.35, rel_tol = 0.02):
                            k[idx] = 6
                            
                        elif a4 >= 1.45:
                            k[idx] = 5
                            
                        elif a6 == 'COWD' or a6 == 'COWD*' or a6 == 'COWD**':
                            k[idx] = 1
                            
                        elif a6 == 'CONeWD' or a6 == 'CONeWD*' or a6 == 'CONeWD**' or a6 == 'CONewD':
                            k[idx] = 2
                            
                        elif a6 == 'ONeWD' or a6 == 'ONeWD*' or a6 == 'ONeWD**':
                            k[idx] = 3
                            
                        elif a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**':
                            k[idx] = 4
                            
                        elif a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**':
                            k[idx] = 5
                            
                    elif a5 > 0.3 and a4 >= 1.45:
                        k[idx] = 5
                        
                    elif a4 > 1.37 and a4 < 1.45 and (a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                        k[idx] = 4
                        
                    elif a4 > 1.37 and a4 < 1.45 and (a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                        k[idx] = 5
                    
                    else:
                        k[idx] = 0
            """
        
            for a1,a2,a3,a4,a5,a6 in zip(m1,z1,f1,c1,e1,r1):

                    if a1 == mass and a2 == metallicity and a3 == overshooting:

                        if a5 <= 0.3:
                            if a4 >= 1.45:
                                k[idx] = 5

                            elif math.isclose(a4, 1.35, rel_tol = 0.02):
                                k[idx] = 6

                            elif a4 < 1.45 and a4 > 1.37 and (
                                a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                                k[idx] = 4

                            elif a4 < 1.45 and a4 > 1.37 and (
                                a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                                k[idx] = 5

                            elif a4 < 1.33 and (a6 == 'COWD' or a6 == 'COWD*' or a6 == 'COWD**'):
                                k[idx] = 1

                            elif a4 < 1.33 and (
                                a6 == 'CONeWD' or a6 == 'CONeWD*' or a6 == 'CONeWD**' or a6 == 'CONewD'):
                                k[idx] = 2

                            elif a4 < 1.33 and (a6 == 'ONeWD' or a6 == 'ONeWD*' or a6 == 'ONeWD**'):
                                k[idx] = 3

                            else:
                                k[idx] = 0

                        else:

                            if a4 >= 1.45:
                                k[idx] = 5

                            elif a4 < 1.45 and a4 > 1.37 and (
                                a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                                k[idx] = 5

                            elif a4 < 1.45 and (
                                a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                                k[idx] = 4

                            else:
                                k[idx] = 0
            
        
    return Z

In [ ]:
from matplotlib.colors import ListedColormap as cmap
from matplotlib.colors import BoundaryNorm as bn
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

myCmap = cmap(['lightgray', 'lightskyblue', 'dodgerblue', 'green', 'purple', 'orange', 'red'])
bounds = [0,1,2,3,4,5,6]
norm = bn(bounds, myCmap.N)

In [ ]:
Z = f(X,Y)


hp.homogenise_plot(fig_width = 15, fig_height = 10, fontsize = 20)

plt.figure()
ax = plt.gca()

img = ax.imshow(Z, cmap = myCmap, interpolation = 'none', origin = 'lower')
ax.add_patch(mpl.patches.Rectangle((8.5, 6.5), 1, 1, hatch = '//', fill = False, snap = False, color = 'brown'))
ax.add_patch(mpl.patches.Rectangle((9.5, 6.5), 1, 1, hatch = '//', fill = False, snap = False, color = 'brown'))

plt.axhline(2.5, c = 'k', linestyle = '--', linewidth = 2)
plt.axhline(5.5, c = 'k', linestyle = '--', linewidth = 2)

#ax.set_xlim([0.0, 27.5])
ax.set_ylim([-0.5, 8])

#ax.xaxis.set_major_locator(plt.MaxNLocator(20))
#ax.yaxis.set_major_locator(plt.MaxNLocator(9))
ax.xaxis.set_minor_locator(plt.MaxNLocator(29))
ax.yaxis.set_minor_locator(plt.MaxNLocator(9))

ax.tick_params(axis = 'both', which = 'major', length=0)


plt.grid(True, which = 'major', axis = 'both', color = 'white')

xlabels = [item.get_text() for item in ax.get_xticklabels(minor = True)]
ylabels = [item.get_text() for item in ax.get_yticklabels(minor = True)]




xlabels[1] = '0.8'
#xlabels[2] = '0.9'
xlabels[3] = '1.0'
#xlabels[4] = '1.1'
xlabels[5] = '1.2'
#xlabels[6] = '1.3'
xlabels[7] = '1.4'
#xlabels[8] = '1.5'
xlabels[9] = '1.6'
#xlabels[10] = '1.7'
xlabels[11] = '1.8'
xlabels[13] = '2.0'
xlabels[15] = '2.2'
xlabels[17] = '2.4'
xlabels[19] = '2.6'
xlabels[21] = '2.8'
xlabels[23] = '3.0'
xlabels[25] = '3.2'
xlabels[27] = '3.4'



ylabels[1] = '(0.0001;0.0)'
ylabels[2] = '(0.0001;0.014)'
ylabels[3] = '(0.0001;0.016)'
ylabels[4] = '(0.001;0.0)'
ylabels[5] = '(0.001;0.014)'
ylabels[6] = '(0.001;0.016)'
ylabels[7] = '(0.02;0.0)'
ylabels[8] = '(0.02;0.014)'
ylabels[9] = '(0.02;0.016)'

# Shift grid by 0.5 in both directions
ax.set_xticks(np.arange(-.5, 27, 1))
ax.set_yticks(np.arange(-.5, 9, 1))


ax.set_xticklabels(xlabels, minor=True, size = 18)
ax.set_xticklabels([])
ax.set_xlabel(r'Initial Mass [M$_{\odot}$]', labelpad=20)


ax.set_yticklabels(ylabels, minor=True, size = 15)
ax.set_yticklabels([])
ax.set_ylabel(r'Metallicity/Overshooting', labelpad=20)

# Colorbar config
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)

cbar = plt.colorbar(img, cmap = myCmap, cax = cax)
cbar.ax.set_yticklabels(['N/A','COWD', 'CONeWD', 'ONeWD', 'ONeSiWD', 'CCSN', 'TNE'], size = 13)

#plt.savefig('parameterSpaceGrid.png')

In [ ]:
x = np.linspace(0.0, 27.0, 28)
y = np.linspace(0.0, 2, 3)

XLOW,YLOW = np.meshgrid(x,y)
XINTER,YINTER = np.meshgrid(x,y)
XHIGH, YHIGH = np.meshgrid(x,y)

In [ ]:
def function(X1,Y1, X2,Y2, X3,Y3):
    
    m1,z1,f1,c1,e1,r1 = [],[],[],[],[],[]
    ZLOW = np.zeros((3, 28))
    ZINTER = np.zeros((3, 28))
    ZHIGH = np.zeros((3, 28))
    
    # Read data from file
    with open('fullCoreGrowthData.csv', 'r') as file:
        next(file)
        reader = csv.reader(file)
        
        for row in reader:
            m1.append(row[0])
            z1.append(row[1])
            f1.append(row[2])
            c1.append(float(row[3]))
            e1.append(float(row[4]))
            r1.append(row[5])
            
            
            
    for i,j,k in list(zip(X1,Y1,ZLOW)):
        
        for idx, value in enumerate(j):
    
            if math.isclose(value,0.0):
                metallicity = '0.0001'
                overshooting = '0.0000'

            elif math.isclose(value,1.0):
                metallicity = '0.0001'
                overshooting = '0.0140'

            elif math.isclose(value,2.0):
                metallicity = '0.0001'
                overshooting = '0.0160'
                
                
            if math.isclose(i[idx], 0.0):
                mass = '0.8000'

            elif math.isclose(i[idx], 1.0):
                mass = '0.9000'

            elif math.isclose(i[idx], 2.0):
                mass = '1.0000'

            elif math.isclose(i[idx], 3.0):
                mass = '1.1000'

            elif math.isclose(i[idx], 4.0):
                mass = '1.2000'

            elif math.isclose(i[idx], 5.0):
                mass = '1.3000'

            elif math.isclose(i[idx], 6.0):
                mass = '1.4000'

            elif math.isclose(i[idx], 7.0):
                mass = '1.5000'

            elif math.isclose(i[idx], 8.0):
                mass = '1.6000'

            elif math.isclose(i[idx], 9.0):
                mass = '1.7000'

            elif math.isclose(i[idx], 10.0):
                mass = '1.8000'

            elif math.isclose(i[idx], 11.0):
                mass = '1.9000'

            elif math.isclose(i[idx], 12.0):
                mass = '2.0000'

            elif math.isclose(i[idx], 13.0):
                mass = '2.1000'

            elif math.isclose(i[idx], 14.0):
                mass = '2.2000'

            elif math.isclose(i[idx], 15.0):
                mass = '2.3000'

            elif math.isclose(i[idx], 16.0):
                mass = '2.4000'

            elif math.isclose(i[idx], 17.0):
                mass = '2.5000'

            elif math.isclose(i[idx], 18.0):
                mass = '2.6000'

            elif math.isclose(i[idx], 19.0):
                mass = '2.7000'

            elif math.isclose(i[idx], 20.0):
                mass = '2.8000'

            elif math.isclose(i[idx], 21.0):
                mass = '2.9000'

            elif math.isclose(i[idx], 22.0):
                mass = '3.0000'

            elif math.isclose(i[idx], 23.0):
                mass = '3.1000'

            elif math.isclose(i[idx], 24.0):
                mass = '3.2000'

            elif math.isclose(i[idx], 25.0):
                mass = '3.3000'

            elif math.isclose(i[idx], 26.0):
                mass = '3.4000'

            elif math.isclose(i[idx], 27.0):
                mass = '3.5000'
                
                
            for a1,a2,a3,a4,a5,a6 in zip(m1,z1,f1,c1,e1,r1):

                if a1 == mass and a2 == metallicity and a3 == overshooting:
                    
                    if a5 <= 0.3:
                        if a4 >= 1.45:
                            k[idx] = 5
                            
                        elif math.isclose(a4, 1.35, rel_tol = 0.02):
                            k[idx] = 6
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                            k[idx] = 4
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                            k[idx] = 5
                            
                        elif a4 < 1.33 and (a6 == 'COWD' or a6 == 'COWD*' or a6 == 'COWD**'):
                            k[idx] = 1
                            
                        elif a4 < 1.33 and (
                            a6 == 'CONeWD' or a6 == 'CONeWD*' or a6 == 'CONeWD**' or a6 == 'CONewD'):
                            k[idx] = 2
                            
                        elif a4 < 1.33 and (a6 == 'ONeWD' or a6 == 'ONeWD*' or a6 == 'ONeWD**'):
                            k[idx] = 3
                            
                        else:
                            k[idx] = 0
                            
                    else:
                        
                        if a4 >= 1.45:
                            k[idx] = 5
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                            k[idx] = 5
                            
                        elif a4 < 1.45 and (
                            a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                            k[idx] = 4
                            
                        else:
                            k[idx] = 0
                            
                        
        

        

    for i,j,k in list(zip(X2,Y2,ZINTER)):
        
        for idx, value in enumerate(j):
    
            if math.isclose(value,0.0):
                metallicity = '0.0010'
                overshooting = '0.0000'

            elif math.isclose(value,1.0):
                metallicity = '0.0010'
                overshooting = '0.0140'

            elif math.isclose(value,2.0):
                metallicity = '0.0010'
                overshooting = '0.0160'
                
                
            if math.isclose(i[idx], 0.0):
                mass = '0.8000'

            elif math.isclose(i[idx], 1.0):
                mass = '0.9000'

            elif math.isclose(i[idx], 2.0):
                mass = '1.0000'

            elif math.isclose(i[idx], 3.0):
                mass = '1.1000'

            elif math.isclose(i[idx], 4.0):
                mass = '1.2000'

            elif math.isclose(i[idx], 5.0):
                mass = '1.3000'

            elif math.isclose(i[idx], 6.0):
                mass = '1.4000'

            elif math.isclose(i[idx], 7.0):
                mass = '1.5000'

            elif math.isclose(i[idx], 8.0):
                mass = '1.6000'

            elif math.isclose(i[idx], 9.0):
                mass = '1.7000'

            elif math.isclose(i[idx], 10.0):
                mass = '1.8000'

            elif math.isclose(i[idx], 11.0):
                mass = '1.9000'

            elif math.isclose(i[idx], 12.0):
                mass = '2.0000'

            elif math.isclose(i[idx], 13.0):
                mass = '2.1000'

            elif math.isclose(i[idx], 14.0):
                mass = '2.2000'

            elif math.isclose(i[idx], 15.0):
                mass = '2.3000'

            elif math.isclose(i[idx], 16.0):
                mass = '2.4000'

            elif math.isclose(i[idx], 17.0):
                mass = '2.5000'

            elif math.isclose(i[idx], 18.0):
                mass = '2.6000'

            elif math.isclose(i[idx], 19.0):
                mass = '2.7000'

            elif math.isclose(i[idx], 20.0):
                mass = '2.8000'

            elif math.isclose(i[idx], 21.0):
                mass = '2.9000'

            elif math.isclose(i[idx], 22.0):
                mass = '3.0000'

            elif math.isclose(i[idx], 23.0):
                mass = '3.1000'

            elif math.isclose(i[idx], 24.0):
                mass = '3.2000'

            elif math.isclose(i[idx], 25.0):
                mass = '3.3000'

            elif math.isclose(i[idx], 26.0):
                mass = '3.4000'

            elif math.isclose(i[idx], 27.0):
                mass = '3.5000'
                
                
            for a1,a2,a3,a4,a5,a6 in zip(m1,z1,f1,c1,e1,r1):

                if a1 == mass and a2 == metallicity and a3 == overshooting:
                    
                    if a5 <= 0.3:
                        if a4 >= 1.45:
                            k[idx] = 5
                            
                        elif math.isclose(a4, 1.35, rel_tol = 0.02):
                            k[idx] = 6
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                            k[idx] = 4
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                            k[idx] = 5
                            
                        elif a4 < 1.33 and (a6 == 'COWD' or a6 == 'COWD*' or a6 == 'COWD**'):
                            k[idx] = 1
                            
                        elif a4 < 1.33 and (
                            a6 == 'CONeWD' or a6 == 'CONeWD*' or a6 == 'CONeWD**' or a6 == 'CONewD'):
                            k[idx] = 2
                            
                        elif a4 < 1.33 and (a6 == 'ONeWD' or a6 == 'ONeWD*' or a6 == 'ONeWD**'):
                            k[idx] = 3
                            
                        else:
                            k[idx] = 0
                            
                    else:
                        
                        if a4 >= 1.45:
                            k[idx] = 5
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                            k[idx] = 5
                            
                        elif a4 < 1.45 and (
                            a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                            k[idx] = 4
                            
                        else:
                            k[idx] = 0
                            
                            
                            
                            
                            
    for i,j,k in list(zip(X3,Y3,ZHIGH)):
        
        for idx, value in enumerate(j):
    
            if math.isclose(value,0.0):
                metallicity = '0.0200'
                overshooting = '0.0000'

            elif math.isclose(value,1.0):
                metallicity = '0.0200'
                overshooting = '0.0140'

            elif math.isclose(value,2.0):
                metallicity = '0.0200'
                overshooting = '0.0160'
                
                
            if math.isclose(i[idx], 0.0):
                mass = '0.8000'

            elif math.isclose(i[idx], 1.0):
                mass = '0.9000'

            elif math.isclose(i[idx], 2.0):
                mass = '1.0000'

            elif math.isclose(i[idx], 3.0):
                mass = '1.1000'

            elif math.isclose(i[idx], 4.0):
                mass = '1.2000'

            elif math.isclose(i[idx], 5.0):
                mass = '1.3000'

            elif math.isclose(i[idx], 6.0):
                mass = '1.4000'

            elif math.isclose(i[idx], 7.0):
                mass = '1.5000'

            elif math.isclose(i[idx], 8.0):
                mass = '1.6000'

            elif math.isclose(i[idx], 9.0):
                mass = '1.7000'

            elif math.isclose(i[idx], 10.0):
                mass = '1.8000'

            elif math.isclose(i[idx], 11.0):
                mass = '1.9000'

            elif math.isclose(i[idx], 12.0):
                mass = '2.0000'

            elif math.isclose(i[idx], 13.0):
                mass = '2.1000'

            elif math.isclose(i[idx], 14.0):
                mass = '2.2000'

            elif math.isclose(i[idx], 15.0):
                mass = '2.3000'

            elif math.isclose(i[idx], 16.0):
                mass = '2.4000'

            elif math.isclose(i[idx], 17.0):
                mass = '2.5000'

            elif math.isclose(i[idx], 18.0):
                mass = '2.6000'

            elif math.isclose(i[idx], 19.0):
                mass = '2.7000'

            elif math.isclose(i[idx], 20.0):
                mass = '2.8000'

            elif math.isclose(i[idx], 21.0):
                mass = '2.9000'

            elif math.isclose(i[idx], 22.0):
                mass = '3.0000'

            elif math.isclose(i[idx], 23.0):
                mass = '3.1000'

            elif math.isclose(i[idx], 24.0):
                mass = '3.2000'

            elif math.isclose(i[idx], 25.0):
                mass = '3.3000'

            elif math.isclose(i[idx], 26.0):
                mass = '3.4000'

            elif math.isclose(i[idx], 27.0):
                mass = '3.5000'
                
                
            for a1,a2,a3,a4,a5,a6 in zip(m1,z1,f1,c1,e1,r1):

                if a1 == mass and a2 == metallicity and a3 == overshooting:
                    
                    if a5 <= 0.3:
                        if a4 >= 1.45:
                            k[idx] = 5
                            
                        elif math.isclose(a4, 1.35, rel_tol = 0.02):
                            k[idx] = 6
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                            k[idx] = 4
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                            k[idx] = 5
                            
                        elif a4 < 1.33 and (a6 == 'COWD' or a6 == 'COWD*' or a6 == 'COWD**'):
                            k[idx] = 1
                            
                        elif a4 < 1.33 and (
                            a6 == 'CONeWD' or a6 == 'CONeWD*' or a6 == 'CONeWD**' or a6 == 'CONewD'):
                            k[idx] = 2
                            
                        elif a4 < 1.33 and (a6 == 'ONeWD' or a6 == 'ONeWD*' or a6 == 'ONeWD**'):
                            k[idx] = 3
                            
                        else:
                            k[idx] = 0
                            
                    else:
                        
                        if a4 >= 1.45:
                            k[idx] = 5
                            
                        elif a4 < 1.45 and a4 > 1.37 and (
                            a6 == 'SiCORE' or a6 == 'SiCORE*' or a6 == 'SiCORE**'):
                            k[idx] = 5
                            
                        elif a4 < 1.45 and (
                            a6 == 'ONeSiWD' or a6 == 'ONeSiWD*' or a6 == 'ONeSiWD**'):
                            k[idx] = 4
                            
                        else:
                            k[idx] = 0
                            
                            
    return ZLOW, ZINTER, ZHIGH

In [ ]:
ZLOW, ZINTER, ZHIGH = function(XLOW,YLOW, XINTER, YINTER, XHIGH, YHIGH)

In [ ]:
hp.homogenise_plot(fig_width = 20, columns = 2, fontsize = 20)


fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex='all', sharey='all')

fig.subplots_adjust(hspace=0.01)

img3 = ax3.imshow(ZLOW, cmap = myCmap, interpolation = 'none', origin = 'lower')
img2 = ax2.imshow(ZINTER, cmap = myCmap, interpolation = 'none', origin = 'lower')
img1 = ax1.imshow(ZHIGH, cmap = myCmap, interpolation = 'none', origin = 'lower')
ax1.add_patch(mpl.patches.Rectangle((8.5, 0.5), 1, 1, hatch = '//', fill = False, snap = False, color = 'k'))
ax1.add_patch(mpl.patches.Rectangle((9.5, 0.5), 1, 1, hatch = '//', fill = False, snap = False, color = 'k'))

ax1.grid(True, which = 'major', axis = 'both', color = 'white')
ax2.grid(True, which = 'major', axis = 'both', color = 'white')
ax3.grid(True, which = 'major', axis = 'both', color = 'white')

ax1.xaxis.set_minor_locator(plt.MaxNLocator(29))
ax1.yaxis.set_minor_locator(plt.MaxNLocator(3))
ax2.yaxis.set_minor_locator(plt.MaxNLocator(3))
ax3.yaxis.set_minor_locator(plt.MaxNLocator(3))

ax1.tick_params(axis = 'both', which = 'major', length=0)
ax2.tick_params(axis = 'both', which = 'major', length=0)
ax3.tick_params(axis = 'both', which = 'major', length=0)

# Shift grid by 0.5 in both directions
ax1.set_xticks(np.arange(-.5, 27, 1))
ax1.set_yticks(np.arange(-.5, 3, 1))

ax2.set_xticks(np.arange(-.5, 27, 1))
ax2.set_yticks(np.arange(-.5, 3, 1))

ax3.set_xticks(np.arange(-.5, 27, 1))
ax3.set_yticks(np.arange(-.5, 3, 1))

# Get labels
xlabels = [item.get_text() for item in ax3.get_xticklabels(minor = True)]
y1labels = [item.get_text() for item in ax1.get_yticklabels(minor = True)]
y2labels = [item.get_text() for item in ax2.get_yticklabels(minor = True)]
y3labels = [item.get_text() for item in ax3.get_yticklabels(minor = True)]

# Create custom labels
xlabels[1] = '0.8'
#xlabels[2] = '0.9'
xlabels[3] = '1.0'
#xlabels[4] = '1.1'
xlabels[5] = '1.2'
#xlabels[6] = '1.3'
xlabels[7] = '1.4'
#xlabels[8] = '1.5'
xlabels[9] = '1.6'
#xlabels[10] = '1.7'
xlabels[11] = '1.8'
xlabels[13] = '2.0'
xlabels[15] = '2.2'
xlabels[17] = '2.4'
xlabels[19] = '2.6'
xlabels[21] = '2.8'
xlabels[23] = '3.0'
xlabels[25] = '3.2'
xlabels[27] = '3.4'

y1labels[1] = r'$f=0.0$'
y1labels[2] = r'$f=0.014$'
y1labels[3] = r'$f=0.016$'

# Set new labels
ax3.set_xticklabels(xlabels, minor=True)
ax3.set_xticklabels([])
ax3.set_xlabel(r'Initial Mass [M$_{\odot}$]', labelpad=25)

ax1.set_yticklabels(y1labels, minor=True)
ax1.set_yticklabels([])




# Colorbar config
#divider = make_axes_locatable(ax1)
#cax = divider.append_axes("right", size="5%", pad=0.2)

#cbar = plt.colorbar(img, cmap = myCmap, cax = cax)
#cbar.ax.set_yticklabels(['N/A','COWD', 'CONeWD', 'ONeWD', 'ONeSiWD', 'CCSN', 'TNE'], size = 13)


fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(img1, cax=cbar_ax)
cbar_ax.set_yticklabels(['N/A','COWD', 'CONeWD', 'ONeWD', 'ONeSiWD', 'CCSN', 'TNE'], size = 15)

#plt.savefig('parameterSpaceRaster.png')